### Let us start our exercise by importing the relevant libraries

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Define the test functions

In [2]:
k = 8

a2 = np.array([0,0.5,1,1.5,2,2.5,3,3.5])
b3 = np.array([6.42,6.42,6.42,6.42,6.42,6.42])

def A1(sm):
    return pd.Series([np.prod(sm.iloc[:,:j+1],axis=1)*(-1)**(j+1) for j in range(k)]).sum()

def A2(sm):
    return pd.Series([(np.abs(4*sm[j]-2)+a2[j])/(1+a2[j]) for j in range(k)]).product()

def A2b(sm,sn):
    return pd.Series([(np.abs(4*(sm[j]+sn[j]-np.modf(sm[j]+sn[j])[1])-2)+a2[j])/(1+a2[j]) for j in range(k)]).product()

def B1(sm):
    return pd.Series([(k-sm[j])/(k-0.5) for j in range(k)]).product()
        
def B2(sm):
    return ((1+1/k)**k)*pd.Series([sm[j]**(1/k) for j in range(k)]).product()
        
def B3(sm):
    return pd.Series([(np.abs(4*sm[j]-2)+b3[j])/(1+b3[j]) for j in range(k)]).product()
        
def C1(sm):
    return pd.Series([np.abs(4*sm[j]-2) for j in range(k)]).product()
        
def C2(sm):
    return sm.product(axis=1)*2**k

functions = [A1,A2]

### Let us produce the sample matrix

In [3]:
p = 13
run = 50

n = 2

df = pd.DataFrame(sobol_seq.i4_sobol_generate(2*k, run*2**p))

### It is finally time to assess the ranks

In [4]:
run_samples = []

MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
for r in range (1):
    run_samples.append(df.iloc[int(r*(len(df)/run)):int((r+1)*(len(df)/run))].reset_index(drop=True))

    sample_Matrices = [run_samples[-1].iloc[:,m*int(len(run_samples[-1].T)/n):
                                            (m+1)*int(len(run_samples[-1].T)/n)].T.reset_index(drop=True).T 
                       for m in range(n)]
    
    mixed_Matrices = []
    for j in range(k):
        mixed_Matrices.append(sample_Matrices[0].copy())
        mixed_Matrices[-1][j]=sample_Matrices[-1][j]
        
    elementary_effects_dic = {f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions}
    elementary_effectsFO_dic = {f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions}
    for f in functions:
        for j in range(k):
            elementary_effects_dic[f.__name__][j] = 0.5*(f(sample_Matrices[0])-f(mixed_Matrices[j]))**2
            elementary_effectsFO_dic[f.__name__][j] = f(sample_Matrices[1])*(f(mixed_Matrices[j])-f(sample_Matrices[0]))
    
        C_T = elementary_effects_dic[f.__name__].expanding(1).mean()
        C_S = elementary_effectsFO_dic[f.__name__].expanding(1).mean()
        C_rT = C_T.rank(method='min',ascending=False,axis=1)
        C_rS = C_S.rank(method='min',ascending=False,axis=1)
        T_sR = C_rT[C_rT!=C_rT.iloc[-1]].dropna(how='all').index[-1]
        S_sR = C_rS[C_rS!=C_rS.iloc[-1]].dropna(how='all').index[-1]
    
        print('Function '+str(f.__name__)+' : First-order-indices rank converges at the '+str(S_sR)+'th row''\n'
              'Function '+str(f.__name__)+' :Total-order-indices rank converges at the '+str(T_sR)+'th row')

Function A1 : First-order-index rank converges at the 3609th row
Function A1 :Total-order-index rank converges at the 763th row
Function A2 : First-order-index rank converges at the 3141th row
Function A2 :Total-order-index rank converges at the 94th row
